In [18]:
import os
import json
import pandas as pd
from tqdm import tqdm

In [14]:
import os
import jsonlines
from tqdm import tqdm

save_names = ['llava_pretrain']
data = {name: [] for name in save_names}
file_path = '/home/mila/l/le.zhang/scratch/datasets/Cambrian-Alignment/jsons/alignment_2.5m.jsonl'
base_path = '/home/mila/l/le.zhang/scratch/datasets/Cambrian-Alignment'

# Count the total number of lines for the progress bar
total_lines = sum(1 for _ in open(file_path))

with jsonlines.open(file_path) as reader:
    for entry in tqdm(reader, total=total_lines, desc="Processing entries"):
        data_split = entry['image'].split('/')[1]
        
        if data_split in save_names:
            if data_split == 'llava_pretrain':
                entry['image'] = entry['image'].replace('llava_pretrain/images', 'sbu558k')
            image_path = os.path.join(base_path, entry['image'])
            image_path = os.path.normpath(image_path)
            
            if os.path.exists(image_path):
                data[data_split].append(entry)
            else:
                break
# count the number of each dataset
for key, entries in data.items():
    print(f"{key}: {len(entries)}")


Processing entries: 100%|██████████| 2513677/2513677 [02:01<00:00, 20663.97it/s] 

llava_pretrain: 558128


In [16]:
data.keys()

dict_keys(['llava_pretrain'])

/home/mila/l/le.zhang/scratch/datasets/Cambrian-Alignment/sbu558k/00120/001200924.jpg

In [19]:
# save as json for each data
for key, entries in data.items():
    save_path = f'/home/mila/l/le.zhang/scratch/datasets/Cambrian-Alignment/jsons/{key}.json'
    with open(save_path, 'w') as f:
        json.dump(entries, f)
    print(f"Saved {len(entries)} entries to {save_path}")

Saved 558128 entries to /home/mila/l/le.zhang/scratch/datasets/Cambrian-Alignment/jsons/llava_pretrain.json


In [ ]:
data['sam'][0]

In [ ]:
import torch
from tqdm import tqdm
def get_linear_input(sequence_output):
    cls_token = sequence_output[:, 0]
    patch_tokens = sequence_output[:, 1:]
    linear_input = torch.cat([cls_token, patch_tokens.mean(dim=1)], dim=1)
    return linear_input
directory = '/home/mila/l/le.zhang/scratch/light_align/data/image_embedding/dinov2-base'
for filename in os.listdir(directory):
    if filename.endswith(".pt"):
        print(os.path.join(directory, filename))
        image_embedding = torch.load(os.path.join(directory, filename))
        image_embedding = get_linear_input(image_embedding)
        torch.save(image_embedding, os.path.join(directory, filename))
    else:
        continue

In [ ]:
import torch
vision_embedding = torch.load('/home/mila/l/le.zhang/scratch/light_align/data/text_embedding/gte-large-en-v1.5/sam/0.pt')
vision_embedding.shape

In [ ]:
text_embedding = torch.load('/home/mila/l/le.zhang/scratch/light_align/data/text_embedding/gte-large-en-v1.5/ALLaVALAION/1.pt')
text_embedding.shape

In [ ]:
model_name = 'all-mpnet-base-v2'
with open(os.path.join('./text_embedding', model_name, 'metadata.json'), 'w') as f:
    json.dump({'encoded_model_name': model_name, 'chunk_size': 5120, 'total_data_num': 558128}, f)

In [ ]:
import numpy as np
a=np.array(4.31)
np.exp(a)

In [ ]:
import glob
import os
from natsort import natsorted
path_list = ['/home/mila/l/le.zhang/scratch/light_align/data/image_embedding/dinov2-large/ALLaVAVFLAN', '/home/mila/l/le.zhang/scratch/light_align/data/image_embedding/dinov2-large/LLaVA558K']
all_files = []
for dir_path in path_list:
    files = glob.glob(os.path.join(dir_path, "*.pt"))
    sorted_files = natsorted(files)
    all_files.extend(sorted_files)
all_files

In [20]:
import os
import json
# read json
with open('/home/mila/l/le.zhang/scratch/datasets/Cambrian-Alignment/jsons/llava_pretrain.json', 'r') as f:
    sharegpt4v = json.load(f)
with open('/home/mila/l/le.zhang/scratch/datasets/Cambrian-Alignment/jsons/sbu558k.json', 'r') as f:
    sbu558k = json.load(f)
with open('/home/mila/l/le.zhang/scratch/datasets/Cambrian-Alignment/jsons/coco.json', 'r') as f:
    coco = json.load(f)




In [21]:
# function to caculate avg length 
def avg_length(data):
    avg = 0
    for entry in data:
        avg += len(entry['conversations'][-1]['value'])
    return avg/len(data)

avg_length(sharegpt4v), avg_length(sbu558k), avg_length(coco)

(824.5308011782244, 54.432029928618526, 816.553433597944)

In [22]:
sharegpt4v[0]

{'id': '001200924',
 'image': './sbu558k/00120/001200924.jpg',
 'conversations': [{'from': 'human',
   'value': '<image>\nWhat are the key elements in this picture?'},
  {'from': 'gpt',
   'value': 'The image is a weather warning poster issued by the National Weather Service for Los Angeles/Oxnard, California on Monday, October 14, 2013 at 8:32 PM PDT. The poster is divided into three sections. \n\nIn the top left section, there\'s a photograph of a desert landscape with palm trees and mountains in the background. The sky above is a clear blue, suggesting a sunny day. \n\nThe top right section of the poster contains text that reads "Gusty Santa Ana Winds & High Fire Danger Possible on Wednesday Fire Weather Watch Wednesday 6 AM - 6 PM L.A. & Ventura County Valleys & Mountains Excluding the San Gabriel and Antelope Valleys Rising Temperatures Gusts up to 45 mph for the mountains & 35 mph for the valleys High Humidity Low Humidity in the 70s to near 80 in the mountains". \n\nThe bottom s